[View in Colaboratory](https://colab.research.google.com/github/vivekster12/Parser/blob/master/NER_spacy.ipynb)

THIS IS FROM SPACY WEBSITE/ TUTORIAL

In [0]:
from __future__ import unicode_literals, print_function

#import plac
import random
from pathlib import Path
import spacy

In [0]:
from google.colab import files
import io

In [3]:
!pip install pathlib

    100% |████████████████████████████████| 51kB 3.4MB/s 
  Running setup.py bdist_wheel for pathlib ... - done
  Stored in directory: /content/.cache/pip/wheels/f9/b2/4a/68efdfe5093638a9918bd1bb734af625526e849487200aa171
Successfully built pathlib


In [2]:
!pip install spacy

  Using cached https://files.pythonhosted.org/packages/24/de/ac14cd453c98656d6738a5669f96a4ac7f668493d5e6b78227ac933c5fd4/spacy-2.0.12.tar.gz
  Using cached https://files.pythonhosted.org/packages/5e/31/c8c1ecafa44db30579c8c457ac7a0f819e8b1dbc3e58308394fff5ff9ba7/murmurhash-0.28.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/be/fc/09684555ce0ee7086675e6be698e4efeb6d9b315fd5aa96bed347572282b/preshed-1.0.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/94/b1/47a88072d0a38b3594c0a638a62f9ef7c742b8b8a87f7b105f7ed720b14b/thinc-6.10.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/9e/9b/62c60d2f5bc135d2aa1d8c8a86aaf84edb719a59c7f11a4316259e61a298/plac-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/c4/79f3409bc710559015464e5f49b9879430d8f87498ecdc335899732e5377/ujson-1.35.tar.gz
  Using cached https://files.pythonhosted.org/packages/6f/78/8b96476f4ae426db71c6e86a8e6a81407f015b34547e442291cd397b18f3/dill-0.2.8

 \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/60/0b/bb/7c2e28db574dbb2358176934eddd32a1c5f838ba0bc23eaaab
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/ca/e5/8b/73706d7232da301838e0bc564367a2f7b2fc8f834228fc8a4b
  Running setup.py bdist_wheel for thinc ... - \

 done
  Stored in directory: /content/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built spacy murmurhash cymem preshed thinc ujson dill regex cytoolz wrapt


In [0]:
# training data
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('Who is Ghengis Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('Who is chingis Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    }),

]

In [25]:
print (type(TRAIN_DATA[0][1]))
# print (TRAIN_DATA[0])

<class 'dict'>


In [28]:
f = open('test', 'wb')
f.write('message')
files.download('test')
f.close()

TypeError: ignored

In [0]:

def main(model=None, output_dir=None, n_iter=10):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
        print ('there')
    # otherwise, get it so we can add labels
    else:
        print ('everywehre')
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        print (type(annotations))
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
 
    
  
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
                print (type(text))
                print (type(annotations))
            print(losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is None:
        print ('here')
        output_dir = Path('Libraries\Documents')
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        x = nlp.to_bytes()
        return x
#         print (type(x))
#         f = open('test', 'wb')
#         f.write(x)
#         files.download('test')
#         f.close()
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text in TEST_DATA:
            print (text, '***')
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [45]:
  # test the saved model
  print("Loading from", output_dir)
  
  nlp2 = spacy.blank('en')
  ner = nlp2.create_pipe('ner')
  nlp2.add_pipe(ner, last=True)
  nlp2 = nlp2.from_bytes(x)
  for text in TEST_DATA:
      print (text, '***')
      doc = nlp2(text)
      print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
      print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from Libraries\Documents
Who is afzal khan? ***
Entities [('afzal khan', 'PERSON')]
Tokens [('Who', '', 2), ('is', '', 2), ('afzal', 'PERSON', 3), ('khan', 'PERSON', 1), ('?', '', 2)]
I like Montreal. ***
Entities [('Montreal', 'LOC')]
Tokens [('I', '', 2), ('like', '', 2), ('Montreal', 'LOC', 3), ('.', '', 2)]
I do not like Paris. ***
Entities [('Paris', 'LOC')]
Tokens [('I', '', 2), ('do', '', 2), ('not', '', 2), ('like', '', 2), ('Paris', 'LOC', 3), ('.', '', 2)]
I have never met Mr.Khan. ***
Entities [('never met', 'PERSON')]
Tokens [('I', '', 2), ('have', '', 2), ('never', 'PERSON', 3), ('met', 'PERSON', 1), ('Mr', '', 2), ('.', '', 2), ('Khan', '', 2), ('.', '', 2)]


In [50]:
x = main()

Created blank 'en' model
there
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
{'ner': 19.01908772374736}
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
{'ner': 19.26106513489099}
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>
<class 'str'>
<class 'dict'>


KeyboardInterrupt: ignored

In [0]:

TEST_DATA = [
    ('Who is afzal khan?'),
    ('I like Montreal.'),
    ('I do not like Paris.'),
    ('I have never met Mr.Khan.')
]